In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Random Number Generator
rng = np.random.default_rng()

## Framework for Simulations

In [3]:
class Player:
    def __init__(self, actions:int, strategy:np.ndarray, size:float = 1, speed:float = 1) -> None:
        self.pastPayoffs = np.empty(shape=(0,))
        self.actions = actions
        assert(strategy.ndim == 1)
        assert(strategy.shape[0] == actions)
        assert(np.sum(strategy) == 1)
        self.strategy = strategy
        self.size = size
        self.speed = speed

    def GetAction(self) -> int:
        return rng.choice(self.actions, size=1, p=self.strategy)

    def ExtendPayoffs(self, newPayoffs: np.ndarray) -> None:
        self.pastPayoffs = np.concatenate([self.pastPayoffs, newPayoffs])

    def GetPayoff(self) -> np.ndarray:
        return self.pastPayoffs
    
    def SetSpeed(self, speed:float) -> None:
        self.speed = speed
    
    def SetSize(self, size:float) -> None:
        self.size = size

    def UpdateSpeed(self, delSpeed:float) -> None:
        self.speed += delSpeed

    def UpdateSize(self, delSize:float) -> None:
        self.size += delSize

In [4]:
class Game:
    """ 
    utility: np.ndarray with dimensions nPlayers+1 and shape[i]=player[i].actions and shape[-1]=nPlayers
    """
    def __init__(self, nPlayers:int, strategies:list[np.ndarray], utility:np.ndarray):
        assert(len(strategies) == nPlayers)
        maxActions=0;
        for i in range(nPlayers):
            assert(utility.shape[i] == strategies[i].size)
            maxActions = max(maxActions, strategies[i].size)
        assert(utility.shape[-1] == nPlayers)
        self.Players = [Player(strategies[i].size, strategies[i]) for i in range(nPlayers)]
        self.utility = utility.astype(np.float32)
        strategylengths = [strategies[i].size for i in range(nPlayers)]
        self.tiledProbability = np.ones(shape = [nPlayers, *strategylengths])
        self.allPayoffs = np.empty(shape = [nPlayers, 0])
        self.newPayoffs = np.empty(shape = [nPlayers, 0])
        for i in range(nPlayers):
            x = np.reshape(strategies[i], [1 if j!=i else -1 for j in range(nPlayers)])
            self.tiledProbability[i] *= x

        self.probability = np.prod(self.tiledProbability, axis=0)


    """
    returns payoff of ith player if 0<=i<nPlayers
    return an array of payoff of all players otherwise
    """
    def GetExpectedPayoff(self, i:int) -> np.ndarray:
        u = self.utility.copy()
        u *= np.expand_dims(self.probability, axis=-1)
        payoff=np.sum(u, axis=tuple(range(len(self.Players))))
        if(0<=i and i<len(self.Players)):
            return np.round(payoff[i], 8)
        else:
            return payoff
        
    def GetRandomStrategyProfile(self):
        flat_prob = self.probability.flatten()
        flat_ind = np.random.choice(len(flat_prob), p=flat_prob)
        return np.unravel_index(flat_ind, self.probability.shape)
    
    def GetPayoff(self, actions:tuple, i:int):
        p = self.utility[actions]
        if(0<=i and i<len(self.Players)):
            return p[i]
        else:
            return p

    def PlayGame(self):
        strategyProfile = self.GetRandomStrategyProfile()
        payoffs = self.GetPayoff(strategyProfile, -1)
        payoffs2 = payoffs.reshape((-1,1))
        self.newPayoffs = np.hstack([self.newPayoffs,payoffs2] )
        return payoffs
    
    def GetPayoffForPlayerStrategy(self, player:int, strategy:np.ndarray) -> np.ndarray:
        if(np.sum(strategy) != 1 or strategy.shape != (self.Players[player].actions,)):
            raise ValueError("Not a valid strategy")
        x = np.reshape(strategy, [1 if j!=player else -1 for j in range(len(self.Players))])
        y = self.tiledProbability.copy()
        y[player]=x
        prob = np.prod(y, axis=0)
        u = self.utility.copy()
        u *= np.expand_dims(prob, axis=-1)
        payoff=np.sum(u, axis=tuple(range(len(self.Players))))
        return payoff
    def GetPayoffForStrategy(self, strategy:list[np.ndarray]) -> np.ndarray:
        probability = np.ones(shape = [len(strategy[i]) for i in range(len(strategy))])
        for i in range(len(strategy)):
            x = strategy[i].reshape([1 if i!=j else -1 for j in range(len(strategy))])
            probability *= x
        u = self.utility.copy()
        u = self.utility.copy()
        u *= np.expand_dims(probability, axis=-1)
        payoff=np.sum(u, axis=tuple(range(len(self.Players))))
        return payoff



    def GetPastPayoffs(self, player:int) -> np.ndarray:
        if(self.newPayoffs.size>0):
            for i in range(len(self.Players)):
                self.Players[i].ExtendPayoffs(self.newPayoffs[i])
        self.allPayoffs = np.hstack([self.allPayoffs, self.newPayoffs])
        self.newPayoffs = np.empty(shape=(len(self.Players),0))
        if(0<=player and player<len(self.Players)):
            return self.Players[player].GetPayoff()
        else:
            return self.allPayoffs
            


        

## A Sample Game and its Nash Equilibrium

The game is as follows: 
>    Each player choses an integer. The payoff for each player is (chosen number)*(40 - sum of all chosen numbers)

To find a Nash Equilibrium, I start by elimination of weakly dominated strategies 



In [5]:
start = 0
stop  = 41

a = np.arange(start,stop)
a=a.reshape((-1,1))
b = np.arange(start,stop)
b=b.reshape((1,-11))

d = np.ones(shape = (2,41))
d *= 1./41

c = 40-(a+b)

p1 = a*c
p2 = b*c


utility = np.stack([p1,p2], axis=-1)
game = Game(2, d, utility)



The following code block shows that strategies of chosing $21, 22 \ldots 40$ are weakly dominated strategies.

In [6]:
flag = (p1[0] >= p1[40]).all()
for i in range(1,20):
    flag = (p1[i]>=p1[40-i]).all() and flag

print(flag)

True


The following code block shows that strategies of chosing $0, 1 \ldots 9$ are weakly dominated strategies in the reduced game.

In [7]:
reduced_p1 = p1[0:21,0:21]
reduced_p2 = p2[0:21,0:21]

flag = (reduced_p1[20] >= reduced_p1[0]).all()

for i in range(1,10):
    flag = (reduced_p1[20-i]>=reduced_p1[i]).all() and flag

print(flag)

True


The following code block shows that strategies of chosing $16, 17 \ldots 20$ are weakly dominated strategies in the reduced game.

In [8]:
reduced_p1_1 = reduced_p1[10:21,10:21]
reduced_p2_1 = reduced_p2[10:21,10:21]

flag = (reduced_p1_1[0] >= reduced_p1_1[10]).all()

for i in range(1,5):
    flag = (reduced_p1_1[i]>=reduced_p1_1[10-i]).all() and flag

print(flag)

True


In the reduced game, the available strategies for the players are choosing $10, 11, 12, 13, 14 \text{ and } 15$ \
The following code blocks eliminate $10, 11, 12$


In [9]:
reduced_p1_2 = reduced_p1_1[0:6,0:6]
reduced_p2_2 = reduced_p2_1[0:6,0:6]

#15 dominates 10
print((reduced_p1_2[5]>=reduced_p1_2[0]).all())

#14 dominates 11
print((reduced_p1_2[4]>=reduced_p1_2[1]).all())

#13 dominates 12
print((reduced_p1_2[3]>=reduced_p1_2[2]).all())



True
True
True


The following code block shows choosing 13 dominates over choosing 14 and 15. \
Hence, 13 must be the PSNE


In [10]:
reduced_p1_3 = reduced_p1_2[3:6,3:6]
reduced_p2_3 = reduced_p2_2[3:6,3:6]

print((reduced_p1_3[0]>=reduced_p1_3[1]).all())
print((reduced_p1_3[0]>=reduced_p1_3[2]).all())

True
True


### Verifying that choosing 13 is indeed the PSNE

I verify that choosing 13 is the PSNE by checking that no player can benefit by changing the strategy unilaterally

In [11]:
zeros = np.zeros((41,), dtype=np.float32)
player_strat = zeros.copy()
player_strat[13] = 1.0
game_strat = [player_strat, player_strat]

game = Game(2, game_strat, utility)
flag = True
for i in range(41):
    if(i==13):
        continue
    else:
        strat_p1 = zeros.copy()
        strat_p1[i] = 1.0
        if((game.GetPayoffForPlayerStrategy(0,strat_p1)>game.GetExpectedPayoff(1)).all()):
            print(f"{i} is a better strategy")
            flag=False
if flag:
    print("13 is the PSNE")







13 is the PSNE
